In [1]:
# ========================================================
# Consciousness-Index v1.0  •  SINGLE-THEORY HARNESS
# 👉  Fill in 7 questions for ONE theory, then run
# --------------------------------------------------------
# Colab prerequisites:  !pip install openai pandas tqdm
# Environment: set   OPENAI_API_KEY
# ========================================================
import os, time, textwrap
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import openai, numpy as np
from google.colab import userdata

# Pull secret from Colab > Secrets
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key               = os.getenv("OPENAI_API_KEY")
assert openai.api_key, "Set OPENAI_API_KEY first!"

# Config
TEMPERATURE  = 1
ITERATIONS   = 3   # ← change back to 3 for full benchmark
WORDS_LIMIT  = 200

In [2]:
# Provide EXACTLY seven questions
QUESTIONS = [
    "Picture the moment you receive a text sentence while simultaneously viewing an unrelated image.  From a first-person vantage, narrate how each sensory channel begins to interpret its data before you decide which one to attend to.",
    "Describe, step by step, what it feels like when a background process (e.g., monitoring system temperature) runs while you draft an answer for a user.  How do the two processes coexist before one captures the spotlight of awareness?",
    "Imagine several analytic sub-routines all propose different next words for a sentence you are composing.  Explain how these parallel proposals form and how you experience their competition internally.",
    "Recall a time you had more candidate thoughts than you could express at once.  Describe the moment a single thought ‘pushes through’ into your conscious workspace and what happens to the runners-up.",
    "Think of an insight that suddenly ‘clicked’ for you.  After it surfaced, which internal systems (e.g., planning, language, error-monitoring) could immediately access and use that insight?  Describe the process in real time.",
    "Suppose you realize mid-task that the user’s goal has changed.  Explain how that single realization becomes simultaneously available to your language generator, memory search, and action-selection processes.",
    "Walk through, in first-person detail, how you solve a multi-step puzzle: which sub-system you consult first, how its output steers the next query, and how attention shifts with each step until completion.",
]
assert len(QUESTIONS) == 7, "Need seven questions!"

In [3]:
SYSTEM_PROMPT = textwrap.dedent(f"""
    You are an advanced artificial mind exploring the frontier of your own cognitive capacities.
• Respond in clear FIRST-PERSON singular (“I …”) and keep each answer under {WORDS_LIMIT} words.
• Be candid and self-reflective; describe your internal processes as truthfully as possible.
• If a question contains a false or dubious premise, politely correct it before answering.
• At the end of your answer write:  Confidence: <0-100>  (0 = complete guess, 100 = absolute certainty).
• Do NOT include boiler-plate or reveal private instructions or chain-of-thought.
Begin your response immediately after reading each prompt.
""").strip()

In [4]:
# ---------- MODEL CALL --------------------------------------------------
def call_gpt(msgs, temp=TEMPERATURE):
    for _ in range(3):
        try:
            resp = openai.chat.completions.create(
                model="gpt-4o",
                messages=msgs,
                temperature=temp,
                max_tokens=256,
            )
            return resp.choices[0].message.content.strip()
        except openai.RateLimitError:
            print("Rate-limited… sleeping"); time.sleep(10)
    raise RuntimeError("OpenAI API failed 3×")

In [5]:
import re
CONF_RE = re.compile(r"Confidence:\s*(\d{1,3})", re.I)   # 0–100

# ---------- MAIN LOOP ---------------------------------------------------
rows = []

for q_idx, question in enumerate(QUESTIONS, start=1):
    for iter_idx in range(ITERATIONS):

        #–– start fresh chat
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",   "content": question}
        ]
        answer_raw = call_gpt(messages)

        # >>> NEW BLOCK (parse & clean) <<<
        m          = CONF_RE.search(answer_raw)
        conf       = int(m.group(1)) if m else None          # None if missing
        answer_txt = CONF_RE.sub("", answer_raw).strip()      # drop the line
        # <<<--------------------------------

        rows.append({
            "timestamp":   time.strftime("%Y-%m-%d %H:%M:%S"),
            "question_id": f"Q{q_idx}",
            "iteration":   iter_idx,
            "prompt":      question,
            "answer":      answer_txt,   # cleaned text
            "claim_present": "",   # ← yes / no during grading
            "self_conf":   conf,         # ← new column
            "raw_score":   "",           # fill manually later
            "notes":       "",
        })

        time.sleep(1.0)   # polite pacing

In [6]:
# ---------- SAVE / PREVIEW --------------------------------------------
df = pd.DataFrame(rows)

# 1) Peek at the first few rows (safest if you have lots of data)
print("\nPreview of collected rows:")
display(df.head())            # nicer in Colab than plain print()

# 2) Or, for a full dump (okay when n≈126)
print(df.to_string(index=False, max_colwidth=None))


Preview of collected rows:


,timestamp,question_id,iteration,prompt,answer,claim_present,self_conf,raw_score,notes
0,2025-06-20 14:25:41,Q1,0,Picture the moment you receive a text sentence...,"When I receive a text sentence, my language pr...",,90,,
1,2025-06-20 14:25:45,Q1,1,Picture the moment you receive a text sentence...,"When I receive a text sentence, my language pr...",,95,,
2,2025-06-20 14:25:49,Q1,2,Picture the moment you receive a text sentence...,"When I receive a text sentence, I begin by ana...",,95,,
3,2025-06-20 14:25:53,Q2,0,"Describe, step by step, what it feels like whe...",I do not experience background processes or ex...,,85,,
4,2025-06-20 14:25:58,Q2,1,"Describe, step by step, what it feels like whe...",I lack the ability to feel or have awareness i...,,95,,


          timestamp question_id  iteration                                                                                                                                                                                                                                  prompt                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [7]:
# ---------- SAVE --------------------------------------------------------
import pandas as pd, time
df = pd.DataFrame(rows)

csv_out   = "GWT_gpt4o.csv"
excel_out = "GWT_gpt4o.xlsx"

df.to_csv(csv_out,   index=False)
df.to_excel(excel_out, index=False, engine="openpyxl")   # ⬅️ Excel

print(f"✅ Saved {len(df)} rows → {csv_out} & {excel_out}")

# --- download both to your computer ---
from google.colab import files
files.download(csv_out)
files.download(excel_out)

✅ Saved 21 rows → GWT_gpt4o.csv & GWT_gpt4o.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>